# The ***GroupBy*** Pattern

We have seen the ***GroupBy*** operator in ***Pandas***, but  this is actually a more general ***design pattern*** that can be utilized in many data analyics frameworks and data access interfaces, e.g. in ***SQL***. 

## GroupBy: general Pattern
<img SRC="IMG/groupby.jpg">

#### GroupBy in SQL:
```
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
ORDER BY COUNT(CustomerID) DESC;
```

#### GroupBy in MongoDB
```
db.BusinessProcess.aggregate({
    "$group": {
        _id: {
            status: "$status",
            type: "$type"
        },
        count: {
            $sum: 1
        }
    }
   })
```


In [1]:
#setup example
import numpy as np
import pandas as pd
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.159256,0.680312
1,a,two,0.342478,0.351336
2,b,one,-0.396289,2.034077
3,b,two,0.300712,-0.053468
4,a,one,-0.023203,-1.013587


In [2]:
#group by key1 
grouped = df.groupby(df['key1'])
grouped #this is now a more complex group object

In [3]:
#and generates a table per group
for name, group in grouped:
    print ("name:", name, "\n",group)

name: a 
   key1 key2     data1     data2
0    a  one  1.159256  0.680312
1    a  two  0.342478  0.351336
4    a  one -0.023203 -1.013587
name: b 
   key1 key2     data1     data2
2    b  one -0.396289  2.034077
3    b  two  0.300712 -0.053468


In [4]:
#access group table
grouped.get_group('b')

,key1,key2,data1,data2
2,b,one,-0.396289,2.034077
3,b,two,0.300712,-0.053468


In [5]:
#get numper of entries (rows) per group
grouped.size()

key1
a    3
b    2
dtype: int64

In [6]:
#get number of group entries by columns
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


#### Think of  grouped DataFrames as 3d objects:

In [7]:
#accessing the "3d" group tables
grouped['data2'].get_group('a')

0    0.680312
1    0.351336
4   -1.013587
Name: data2, dtype: float64

In [8]:
grouped.get_group('a')['data2']

0    0.680312
1    0.351336
4   -1.013587
Name: data2, dtype: float64

### Group by external keys

In [9]:
#define external key years as numpy array
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([years]).mean()


2005    0.600815
2006   -0.209746
Name: data1, dtype: float64

### Group by functions

In [10]:
#sort by column and retun top n
def top(df, n=5, column='data1'):
    return df.sort_values(by=column)[-n:]

df.groupby(df['key1']).apply(top,n=5)

key1 key2     data1     data2
key1                                
a    4    a  one -0.023203 -1.013587
     1    a  two  0.342478  0.351336
     0    a  one  1.159256  0.680312
b    2    b  one -0.396289  2.034077
     3    b  two  0.300712 -0.053468

### Group-wise aggregation (apply)
<img SRC="IMG/groupby.jpg">

#### Typical build in aggregation functions:
* sum
* mean
* max / min
* quantile
* ...

In [11]:
#aggregate over the groups
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


In [12]:
grouped.sum()

,data1,data2
key1,,
a,1.478531,0.018062
b,-0.095577,1.980609


#### Custom Aggregation Functions

In [13]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.182459,1.693898
b,0.697000,2.087545


#### Multiple aggregations

In [14]:
#just call a list of function
grouped.agg([peak_to_peak, 'mean', 'median'])

data1                            data2                    
     peak_to_peak      mean    median peak_to_peak      mean    median
key1                                                                  
a        1.182459  0.492844  0.342478     1.693898  0.006021  0.351336
b        0.697000 -0.047788 -0.047788     2.087545  0.990304  0.990304

#### Suppressing the Group Keys 

In [15]:
df.groupby(df['key1']).apply(top,n=2)

key1 key2     data1     data2
key1                                
a    1    a  two  0.342478  0.351336
     0    a  one  1.159256  0.680312
b    2    b  one -0.396289  2.034077
     3    b  two  0.300712 -0.053468

In [16]:
df.groupby(df['key1'], group_keys=False).apply(top,n=2)

,key1,key2,data1,data2
1,a,two,0.342478,0.351336
0,a,one,1.159256,0.680312
2,b,one,-0.396289,2.034077
3,b,two,0.300712,-0.053468


## More  Exercises in the Lab session... 